In [19]:
!ls ga-customer-revenue-prediction

sample_submission.csv    test.csv                 train.csv
sample_submission_v2.csv test_v2.csv              train_v2.csv


In [21]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

def load_df(csv_path='ga-customer-revenue-prediction/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

print(os.listdir("ga-customer-revenue-prediction"))

['sample_submission.csv', 'sample_submission_v2.csv', 'test.csv', 'test_v2.csv', 'train.csv', 'train_v2.csv']


In [23]:
df = load_df()
df.shape

Loaded train.csv. Shape: (903653, 55)


(903653, 55)

In [24]:
df.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,...,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 55 columns):
channelGrouping                                      903653 non-null object
date                                                 903653 non-null int64
fullVisitorId                                        903653 non-null object
sessionId                                            903653 non-null object
socialEngagementType                                 903653 non-null object
visitId                                              903653 non-null int64
visitNumber                                          903653 non-null int64
visitStartTime                                       903653 non-null int64
device.browser                                       903653 non-null object
device.browserSize                                   903653 non-null object
device.browserVersion                                903653 non-null object
device.deviceCategory                                9036

In [91]:
dff = df[['channelGrouping', 'date', 'visitStartTime', 'fullVisitorId', 'visitNumber', 'device.browser',
 'device.deviceCategory', 'device.isMobile', 
 'device.operatingSystem', 'geoNetwork.continent', 'geoNetwork.country',
 'geoNetwork.subContinent', 'totals.transactionRevenue']]

In [92]:
dff.columns = ['channelGrouping', 'date', 'visitStartTime', 'fullVisitorId', 'visitNumber', 'browser',
 'deviceCategory', 'isMobile', 
 'operatingSystem', 'continent', 'country',
 'subContinent', 'transactionRevenue']

In [93]:
dff.head()

,channelGrouping,date,visitStartTime,fullVisitorId,visitNumber,browser,deviceCategory,isMobile,operatingSystem,continent,country,subContinent,transactionRevenue
0,Organic Search,20160902,1472830385,1131660440785968503,1,Chrome,desktop,False,Windows,Asia,Turkey,Western Asia,NaN
1,Organic Search,20160902,1472880147,377306020877927890,1,Firefox,desktop,False,Macintosh,Oceania,Australia,Australasia,NaN
2,Organic Search,20160902,1472865386,3895546263509774583,1,Chrome,desktop,False,Windows,Europe,Spain,Southern Europe,NaN
3,Organic Search,20160902,1472881213,4763447161404445595,1,UC Browser,desktop,False,Linux,Asia,Indonesia,Southeast Asia,NaN
4,Organic Search,20160902,1472822600,27294437909732085,2,Chrome,mobile,True,Android,Europe,United Kingdom,Northern Europe,NaN


In [94]:
dff.to_csv('marketing_table.csv', sep=';', index=False)